In [1]:
import warnings
import data_call
#ignore all the warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import plotly.colors as pc

def plot_labeled_2d_arrays_subplots(array_list, label_list=None, title="2D Labeled Arrays in Subplots"):
    """
    Plot a list of 2D arrays in a grid of subplots with optional labels (for coloring).
    Legend appears on the right and can toggle label visibility across all subplots.
    """
    n = len(array_list)
    cols = min(5, n)
    rows = math.ceil(n / cols)

    print("here")


    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=[f"Plot {i+1}" for i in range(n)],
        horizontal_spacing=0.04,
        vertical_spacing=0.08,
    )

    # Prepare color mapping
    if label_list is not None:
        all_labels = np.concatenate(label_list)
        unique_labels = np.unique(all_labels)
        color_seq = pc.qualitative.Set1 + pc.qualitative.Set2 + pc.qualitative.Dark2
        color_map = {label: color_seq[i % len(color_seq)] for i, label in enumerate(unique_labels)}
    else:
        unique_labels = []
        color_map = {}

    # Track which label's legend has already been shown
    legend_shown = set()

    for i, arr in enumerate(array_list):
        row = i // cols + 1
        col = i % cols + 1

        if label_list is not None:
            labels = label_list[i]
            for label in np.unique(labels):
                mask = (labels == label)
                fig.add_trace(
                    go.Scatter(
                        x=arr[mask, 0],
                        y=arr[mask, 1],
                        mode="markers",
                        marker=dict(color=color_map[label], size=4),
                        name=f"Label {label}",
                        legendgroup=str(label),
                        showlegend=(label not in legend_shown),
                    ),
                    row=row,
                    col=col
                )
                legend_shown.add(label)
        else:
            fig.add_trace(
                go.Scatter(
                    x=arr[:, 0],
                    y=arr[:, 1],
                    mode="markers",
                    marker=dict(color="blue", size=4),
                    showlegend=False
                ),
                row=row,
                col=col
            )

    fig.update_layout(
        title=title,
        template="plotly_white",
        height=250 * rows,
        autosize=True,
        margin=dict(l=20, r=180, t=60, b=40),
        legend=dict(
            title="Labels",
            x=1.01,
            y=1,
            xanchor="left",
            yanchor="top",
            traceorder="normal",
            itemsizing="constant",
            borderwidth=1
        )
    )


    return fig

import numpy as np
import plotly.express as px

def plot_embedding(embedding, labels=None):
    """
    Plot a 2D embedding with optional labels for coloring.

    Args:
        embedding (ndarray): shape (n_samples, 2)
        labels (array-like, optional): labels for coloring points
    """
    if labels is None:
        labels = np.zeros(embedding.shape[0])  # default: all same

    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color=labels.astype(str),  # convert to string for discrete legend
        title="2D Embedding Visualization",
        labels={"x": "Dim 1", "y": "Dim 2", "color": "Label"}
    )

    fig.update_layout(
        width=800,
        height=500,
        title="Custom size Plotly figure"
    )

    fig.show()


In [3]:
import data_call
d_name='Muraro'
X,label=data_call.get_dataset(d_name, mode='PCA')
X_cg,label_new=data_call.get_dataset(d_name, mode='CG')

if type(X_cg) is not np.ndarray:
    X_cg=X_cg.toarray()


print(X.shape,label.shape)
print(type(X),type(X_cg))
print(X_cg.shape,label_new.shape)


hmap={}
t=0
for i in sorted(set(label)):
    hmap[i]=t
    t+=1
new_label= [hmap[i] for i in label]
y= new_label
true_k=len(set(y))

(2122, 50) (2122,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2122, 50) (2122,)


In [4]:
from cplearn.corespect import Corespect
corespect = Corespect(X)
core = corespect.find_core(core_fraction=0.15)
#{"k": true_k}
cluster_core = corespect.cluster_core(core, cluster_algo='louvain',cluster_algo_params={"ng_num": 20,"resolution":1})
propagated_data = corespect.propagate_labels(cluster_core, propagate_algo='adaptive_majority', propagate_algo_params={"ng_num": 20})

Obtained vertex ranking using FlowRank
Clustered core using louvain
Number of clusters found in the core: 11
Remaining nodes in this round 1804
check here 258
Stage, eps, n1, n0 0 0 576 318
Remaining nodes in this round 1546
check here 175
Stage, eps, n1, n0 0 0 751 576
Remaining nodes in this round 1371
check here 87
Stage, eps, n1, n0 0 0 838 751
Remaining nodes in this round 1284
check here 59
Stage, eps, n1, n0 0 0 897 838
Remaining nodes in this round 1225
check here 49
Stage, eps, n1, n0 0 0 946 897
Remaining nodes in this round 1176
check here 35
Stage, eps, n1, n0 0 0 981 946
Remaining nodes in this round 1141
check here 31
Stage, eps, n1, n0 0 0 1012 981
Remaining nodes in this round 1110
check here 32
Stage, eps, n1, n0 0 0 1044 1012
Remaining nodes in this round 1078
check here 27
Stage, eps, n1, n0 0 0 1071 1044
Remaining nodes in this round 1051
check here 22
Stage, eps, n1, n0 0 0 1093 1071
Remaining nodes in this round 1029
check here 37
Stage, eps, n1, n0 0 0 1130 1093


In [5]:
layers,labels_for_layer=propagated_data.get_layers_and_labels()
print([len(ell) for ell in layers])

[318, 880, 896]


In [7]:
from cplearn.coremap_v2 import Coremap

cmap=Coremap(X,round_info=propagated_data.round_info,labels=propagated_data.final_labels)
dict_viz=cmap.anchored_map()



/Users/chandrasekharmukherjee/.pyenv/versions/3.9.6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


test2
(2122, 15) (2122, 15) (2122, 15)
2.583797317830043e-07 1.0
0.05044614296566009 0.07849722313196696 0.10253925128898923 0.12773429069185058 0.15854204816036122 0.19982700858453342 0.263463409013346 0.38034587707124545 0.6802586152563006 



OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.




(514,) (514, 2) (514,)


(514,) (514, 2) (514,)


(514,) (514, 2) (514,)


In [ ]:
from cplearn.visualizer import Visualizer as viz

fig = viz(corespect,mode='layerwise').fig
#Save fig as html
fig.show()

In [ ]:
import numpy as np
from cplearn.corespect import Corespect
from cplearn.visualizer import Visualizer

X = np.random.randn(500, 20)
corespect = Corespect(X)
core = corespect.find_core(core_fraction=0.15)
cluster_core = corespect.cluster_core(
    core,
    cluster_algo="louvain",
    cluster_algo_params={"ng_num": 20, "resolution": 1}
)
propagated_data = corespect.propagate_labels(
    cluster_core,
    propagate_algo="adaptive_majority",
    propagate_algo_params={"ng_num": 20}
)

print(set(propagated_data.round_info))

layers, labels = propagated_data.get_layers_and_labels()
fig = Visualizer(corespect).fig
fig.show()
